In [25]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/lynge/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477,

In [26]:
sta1_RA_WORLD = 174.77413750 - 9*10**(-5) #HD 101274
sta1_DEC_WORLD = -37.75375000 + 6.4*10**(-5)

sta2_RA_WORLD = 174.82283333 - 0.15/3600. #Tycho-2 7740-22-1
sta2_DEC_WORLD = -37.76239167

sta3_RA_WORLD = 174.79553333 #174.79553250 #Tycho-2 7740-529-1
sta3_DEC_WORLD = -37.83470278 #-37.83470333

k_g = 0.16 #https://www.eso.org/sci/observing/tools/Extinction.html
k_r = 0.08
k_i = 0.02
k_z = 0.01

#mag_refsta1_g = 13.291
#mag_refsta1_r = 12.843
#mag_refsta1_i = 12.668
#mag_refsta1_z = 12.593
mag_refsta1_g = 11.689
mag_refsta1_r = 11.491
mag_refsta1_i = 11.465
mag_refsta1_z = 11.500

mag_refsta2_g = 10.166
mag_refsta2_r = 10.229
mag_refsta2_i = 10.347
mag_refsta2_z = 10.456

mag_refsta3_g = 13.291
mag_refsta3_r = 12.843
mag_refsta3_i = 12.668
mag_refsta3_z = 12.593

mag_refsta4_g = 11.861
mag_refsta4_r = 11.562
mag_refsta4_i = 11.460
mag_refsta4_z = 11.430

#refsta1_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525_E_21_10_17
#refsta1_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree 

#refsta2_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525_E_21_10_17
#refsta2_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree 

refsta1_RA = SkyCoord('00:55:10 00:43:13', unit=(u.hourangle, u.deg)).ra.degree #SA092_342
refsta1_DEC = SkyCoord('00:55:10 00:43:13', unit=(u.hourangle, u.deg)).dec.degree 

refsta2_RA = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).ra.degree #SA100_241
refsta2_DEC = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).dec.degree

refsta3_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525-E
refsta3_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree

refsta4_RA = SkyCoord('01:54:38 00:43:01', unit=(u.hourangle, u.deg)).ra.degree #SA093_317
refsta4_DEC = SkyCoord('01:54:38 00:43:01', unit=(u.hourangle, u.deg)).dec.degree 

In [27]:
QUASAR1_g = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/g')]

QUASAR2_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/g')]

QUASAR3_g = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/g')]

QUASAR4_g = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/g')]

refsta1_g = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/g')]

refsta2_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g')]

refsta3_g = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-P525-E/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-P525-E/g')]

refsta4_g = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/g')]

In [28]:
QUASAR1_r = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/r')]

QUASAR2_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/r')]

QUASAR3_r = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/r')]

QUASAR4_r = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/r')]

refsta1_r = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/r')]

refsta2_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r')]

refsta3_r = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-P525-E/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-P525-E/r')]

refsta4_r = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/r')]

In [29]:
QUASAR1_i = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/i')]

QUASAR2_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/i')]

QUASAR3_i = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/i')]

QUASAR4_i = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/i')]

refsta1_i = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/i')]

refsta2_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i')]

refsta3_i = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-P525-E/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-P525-E/i')]

refsta4_i = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/i')]

In [30]:
QUASAR1_z = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-NGC3783/z')]

QUASAR2_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-NGC3783/z')]

QUASAR3_z = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-NGC3783/z')]

QUASAR4_z = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-NGC3783/z')]

refsta1_z = [os.path.join('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/13-11-17-SA092_342/z')]

refsta2_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z')]

refsta3_z = [os.path.join('/media/lynge/LACIE/Calibration/07-12-17-P525-E/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/07-12-17-P525-E/z')]

refsta4_z = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/z')]

In [31]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [32]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [33]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [34]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [35]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [36]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [37]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":5, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [38]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 20. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 20. and \
            DECmain - 20. < out['table']['YMODEL_IMAGE'][j] < DECmain + 20.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [39]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    aperture1 = 0.0012
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        #print 'Frame nr. ', i, compare1[i]
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), com_EXPTIME.append(header['EXPTIME'])
        else:
            print a1
            com_flu.append(float('nan')), com_AIRMASS.append(float('nan')), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        #print 'Frame nr. ', i, start[i]
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), star_EXPTIME.append(header['EXPTIME'])
        else:
            star_flu.append(float('nan')), star_AIRMASS.append(float('nan')), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu/com_EXPTIME
    star_flu = star_flu/star_EXPTIME
    com_flux = np.nanmean(com_flu)
    com_AIRMASS_num = np.nanmean(com_AIRMASS)
    star_AIRMASS_num = np.nanmean(star_AIRMASS)
    mag = -(star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/float(com_flux))
    #print mag
    return mag

In [40]:
com1_1 = findmag(QUASAR1_g,refsta1_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com2_1 = findmag(QUASAR1_g,refsta1_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com3_1 = findmag(QUASAR1_g,refsta1_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)

com1_2 = findmag(QUASAR2_g,refsta2_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com2_2 = findmag(QUASAR2_g,refsta2_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com3_2 = findmag(QUASAR2_g,refsta2_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)

com1_3 = findmag(QUASAR3_g,refsta3_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)
com2_3 = findmag(QUASAR3_g,refsta3_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)
com3_3 = findmag(QUASAR3_g,refsta3_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)

com1_4 = findmag(QUASAR4_g,refsta4_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_g,k_g)
com2_4 = findmag(QUASAR4_g,refsta4_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_g,k_g)
com3_4 = findmag(QUASAR4_g,refsta4_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_g,k_g)

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


nan
nan
nan
nan
nan
nan


In [41]:
print 'g-filter'
print 'Star 1'
print com1_1,com1_2,com1_3,com1_4
print np.nanmean((com1_1,com1_2,com1_3,com1_4)), np.nanstd((com1_1,com1_2,com1_3,com1_4))

print 'Star 2'
print com2_1,com2_2,com2_3,com2_4
print np.nanmean((com2_1,com2_2,com2_3,com2_4)), np.nanstd((com2_1,com2_2,com2_3,com2_4))

print 'Star 3'
print com3_1,com3_2,com3_3,com3_4
print np.nanmean((com3_1,com3_2,com3_3,com3_4)), np.nanstd((com3_1,com3_2,com3_3,com3_4))

g-filter
Star 1
[10.12734078 10.13449338 10.11181307] [10.17439899 10.21911698 10.18405534] [10.34798295 10.34260882 10.3773328 ] [10.35964016 10.36747876 10.32726762]
10.256127471349773 0.10174859240920515
Star 2
[13.18958658 13.1791885  13.16517669] [13.34648401 13.33137301 13.32332314] [nan nan nan] [13.22662246 13.22199187 13.23756879]
13.246812782086913 0.06545578110948845
Star 3
[nan nan nan] [nan nan nan] [nan nan nan] [12.33580651 12.33222147 12.34380855]
12.337278843758272 0.004843614468120021


In [42]:
com1_1 = findmag(QUASAR1_r,refsta1_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com2_1 = findmag(QUASAR1_r,refsta1_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com3_1 = findmag(QUASAR1_r,refsta1_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)

com1_2 = findmag(QUASAR2_r,refsta2_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com2_2 = findmag(QUASAR2_r,refsta2_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com3_2 = findmag(QUASAR2_r,refsta2_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)

com1_3 = findmag(QUASAR3_r,refsta3_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)
com2_3 = findmag(QUASAR3_r,refsta3_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)
com3_3 = findmag(QUASAR3_r,refsta3_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)

com1_4 = findmag(QUASAR4_r,refsta4_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_r,k_r)
com2_4 = findmag(QUASAR4_r,refsta4_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_r,k_r)
com3_4 = findmag(QUASAR4_r,refsta4_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_r,k_r)

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


nan
nan
nan
nan
nan
nan
nan
nan
nan


In [43]:
print 'r-filter'
print 'Star 1'
print com1_1,com1_2,com1_3,com1_4
print np.nanmean((com1_1,com1_2,com1_3,com1_4)), np.nanstd((com1_1,com1_2,com1_3,com1_4))

print 'Star 2'
print com2_1,com2_2,com2_3,com2_4
print np.nanmean((com2_1,com2_2,com2_3,com2_4)), np.nanstd((com2_1,com2_2,com2_3,com2_4))

print 'Star 3'
print com3_1,com3_2,com3_3,com3_4
print np.nanmean((com3_1,com3_2,com3_3,com3_4)), np.nanstd((com3_1,com3_2,com3_3,com3_4))

r-filter
Star 1
[10.26258289 10.27416082 10.2545437 ] [10.29688838 10.34264915 10.31156038] [10.32998581 10.32350662 10.35546914] [10.41670144 10.42397164 10.38837876]
10.331699894364963 0.05428719082870315
Star 2
[12.73873543 12.74167375 12.74307468] [12.80289564 12.79777035 12.79385682] [nan nan nan] [12.89127447 12.88476608 12.89301877]
12.809673997948494 0.061258485403535
Star 3
[nan nan nan] [nan nan nan] [nan nan nan] [12.09129556 12.09610372 12.09880682]
12.095402033733349 0.0031063422533876913


In [44]:
com1_1 = findmag(QUASAR1_i,refsta1_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com2_1 = findmag(QUASAR1_i,refsta1_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com3_1 = findmag(QUASAR1_i,refsta1_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)

com1_2 = findmag(QUASAR2_i,refsta2_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com2_2 = findmag(QUASAR2_i,refsta2_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com3_2 = findmag(QUASAR2_i,refsta2_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)

com1_3 = findmag(QUASAR3_i,refsta3_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)
com2_3 = findmag(QUASAR3_i,refsta3_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)
com3_3 = findmag(QUASAR3_i,refsta3_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)

com1_4 = findmag(QUASAR4_i,refsta4_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_i,k_i)
com2_4 = findmag(QUASAR4_i,refsta4_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_i,k_i)
com3_4 = findmag(QUASAR4_i,refsta4_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_i,k_i)

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


nan
nan
nan


In [45]:
print 'i-filter'
print 'Star 1'
print com1_1,com1_2,com1_3,com1_4
print np.nanmean((com1_1,com1_2,com1_3,com1_4)), np.nanstd((com1_1,com1_2,com1_3,com1_4))

print 'Star 2'
print com2_1,com2_2,com2_3,com2_4
print np.nanmean((com2_1,com2_2,com2_3,com2_4)), np.nanstd((com2_1,com2_2,com2_3,com2_4))

print 'Star 3'
print com3_1,com3_2,com3_3,com3_4
print np.nanmean((com3_1,com3_2,com3_3,com3_4)), np.nanstd((com3_1,com3_2,com3_3,com3_4))

i-filter
Star 1
[10.37416715 10.39055067 10.36201999] [10.38226864 10.37518888 10.37242866] [10.53245173 10.52846745 10.56415102] [10.65474778 10.66707215 10.55709005]
10.480050347703829 0.11138773576377806
Star 2
[12.67789953 12.67953387 12.675714  ] [12.71799914 12.71607033 12.71319643] [nan nan nan] [12.74388829 12.73900448 12.74577884]
12.71212054422569 0.026821506050065776
Star 3
[nan nan nan] [nan nan nan] [nan nan nan] [11.79749603 11.79916998 11.79979329]
11.798819767949368 0.0009699973541284968


In [46]:
com1_1 = findmag(QUASAR1_z,refsta1_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com2_1 = findmag(QUASAR1_z,refsta1_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com3_1 = findmag(QUASAR1_z,refsta1_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)

com1_2 = findmag(QUASAR2_z,refsta2_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com2_2 = findmag(QUASAR2_z,refsta2_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com3_2 = findmag(QUASAR2_z,refsta2_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)

com1_3 = findmag(QUASAR3_z,refsta3_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)
com2_3 = findmag(QUASAR3_z,refsta3_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)
com3_3 = findmag(QUASAR3_z,refsta3_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)

com1_4 = findmag(QUASAR4_z,refsta4_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_z,k_z)
com2_4 = findmag(QUASAR4_z,refsta4_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_z,k_z)
com3_4 = findmag(QUASAR4_z,refsta4_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta4_RA,refsta4_DEC,mag_refsta4_z,k_z)

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [47]:
print 'z-filter'
print 'Star 1'
print com1_1,com1_2,com1_3,com1_4
print np.nanmean((com1_1,com1_2,com1_3,com1_4)), np.nanstd((com1_1,com1_2,com1_3,com1_4))

print 'Star 2'
print com2_1,com2_2,com2_3,com2_4
print np.nanmean((com2_1,com2_2,com2_3,com2_4)), np.nanstd((com2_1,com2_2,com2_3,com2_4))

print 'Star 3'
print com3_1,com3_2,com3_3,com3_4
print np.nanmean((com3_1,com3_2,com3_3,com3_4)), np.nanstd((com3_1,com3_2,com3_3,com3_4))

z-filter
Star 1
[10.49423704 10.48521835 10.48495366] [10.56673797 10.55799205 10.55886414] [10.14989767 10.14620713 10.14147775] [10.43845668 10.43314451 10.44586658]
10.408587794488303 0.15783766202803912
Star 2
[12.87596135 12.86367915 12.84677882] [13.24017413 13.19485024 13.16877261] [nan nan nan] [12.64194419 12.63658387 12.66476419]
12.903723171841698 0.2287199552345118
Star 3
[nan nan nan] [nan nan nan] [nan nan nan] [11.63860153 11.63695164 11.63608956]
11.637214244838674 0.0010421847092430046


In [48]:
HD101274_1 = findmag(NGC3783_r,P_r,sta1_RA_WORLD,sta1_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_22_1 = findmag(NGC3783_r,P_r,sta2_RA_WORLD,sta2_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_529_1 = findmag(NGC3783_r,P_r,sta3_RA_WORLD,sta3_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)




NameError: name 'NGC3783_r' is not defined

In [ ]:
print 'r-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
HD101274_1 = findmag(NGC3783_i,P_i,sta1_RA_WORLD,sta1_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_22_1 = findmag(NGC3783_i,P_i,sta2_RA_WORLD,sta2_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_529_1 = findmag(NGC3783_i,P_i,sta3_RA_WORLD,sta3_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)




In [ ]:
print 'i-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
sta2_RA_WORLD = 174.82083333
sta2_DEC_WORLD = -37.76194444

HD101274_1 = findmag(NGC3783_z,P_z,sta1_RA_WORLD,sta1_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_22_1 = findmag(NGC3783_z,P_z,sta2_RA_WORLD,sta2_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_529_1 = findmag(NGC3783_z,P_z,sta3_RA_WORLD,sta3_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)




In [ ]:
print 'z-filter'
print HD101274_1
print np.nanmean(HD101274_1,), np.nanstd(HD101274_1)

print Tyc_22_1
print np.nanmean(Tyc_22_1), np.nanstd(Tyc_22_1)

print Tyc_529_1
print np.nanmean(Tyc_529_1), np.nanstd(Tyc_529_1)